In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

/home/marcin/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/marcin/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/marcin/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/marcin/anaconda3/envs/ml/lib/python3.7/site-packages

## Datasets.

In [2]:
feature_names = ['TC_COMPRESSOR_STATE', 
#                  'TC_UPPER_SOURCE_PUMP_STATE',
                 'WORK_TIMER', 
                 'TC_CURRENT_FLOW',
                 'TC_CURRENT_POWER_CONSUMPTION',
#                  'TC_COMPRESSOR_OIL_TEMP',
                 'TC_EVD_CONDENSING_PRESSURE']

path = '../data_for_models/data.csv'
df = pd.read_csv(path)
df = df[feature_names]
print("Database: {}".format(df.shape))
# df = df.dropna()
df = df.fillna(method='ffill')
print("Database after fill nan: {}".format(df.shape))

Database: (225676, 5)
Database after fill nan: (225676, 5)


### Train and test dataset.

In [3]:
train_dataset = df[feature_names].sample(frac=0.8,random_state=0)
test_dataset = df[feature_names].drop(train_dataset.index)

train_y = train_dataset.pop('TC_EVD_CONDENSING_PRESSURE')
test_y = test_dataset.pop('TC_EVD_CONDENSING_PRESSURE')

### Normalization.

In [4]:
def normalization(data, columns):
    scaler = MinMaxScaler()
    for col in columns:
        norm = scaler.fit_transform(data[col].values.reshape(-1, 1))
        data[col] = norm
    return data
        
train_dataset = normalization(train_dataset, ['TC_COMPRESSOR_STATE',
#                                               'TC_UPPER_SOURCE_PUMP_STATE',
                                              'TC_CURRENT_FLOW',
                                              'TC_CURRENT_POWER_CONSUMPTION',
                                              'WORK_TIMER',
#                                               'TC_COMPRESSOR_OIL_TEMP'
                                             ])
test_dataset = normalization(test_dataset, ['TC_COMPRESSOR_STATE', 
#                                             'TC_UPPER_SOURCE_PUMP_STATE',
                                            'TC_CURRENT_FLOW',
                                            'TC_CURRENT_POWER_CONSUMPTION',
                                            'WORK_TIMER',
#                                             'TC_COMPRESSOR_OIL_TEMP'
                                           ])

### Model

In [5]:
model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[len(train_dataset.columns)]),
    layers.Dropout(0.5),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

# optimizer = Adam(learning_rate=0.001, beta_1 = 0.9, beta_2=0.999, amsgrad=False)
optimizer = RMSprop(learning_rate=0.005, rho=0.9)
model.compile(loss='mse', optimizer=optimizer, metrics =[metrics.mae])
model.summary()

W0120 14:08:04.217292 140689822590784 deprecation.py:506] From /home/marcin/anaconda3/envs/ml/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                160       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9

### Train

In [6]:
epochs = 50
history = model.fit(train_dataset, 
                    train_y, 
                    epochs=epochs, 
                    validation_split=0.3, 
                    verbose=1, 
                    batch_size=64)

Train on 126378 samples, validate on 54163 samples
Epoch 1/50
126378/126378 [==============================] - 15s 119us/sample - loss: 3409.9591 - mean_absolute_error: 47.3442 - val_loss: 1202.7326 - val_mean_absolute_error: 29.5487
Epoch 2/50
126378/126378 [==============================] - 16s 127us/sample - loss: 2446.2265 - mean_absolute_error: 39.9074 - val_loss: 1125.4989 - val_mean_absolute_error: 28.1384
Epoch 3/50
126378/126378 [==============================] - 19s 151us/sample - loss: 2197.2554 - mean_absolute_error: 37.8665 - val_loss: 969.7121 - val_mean_absolute_error: 25.2014
Epoch 4/50
126378/126378 [==============================] - 22s 176us/sample - loss: 2000.0320 - mean_absolute_error: 36.1883 - val_loss: 1258.4379 - val_mean_absolute_error: 30.2503
Epoch 5/50
126378/126378 [==============================] - 18s 144us/sample - loss: 1832.5874 - mean_absolute_error: 34.5649 - val_loss: 1037.3946 - val_mean_absolute_error: 26.4918
Epoch 6/50
126378/126378 [=========

KeyboardInterrupt: 

In [ ]:
# summarize history for loss
plt.figure(figsize=(15,5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(test_dataset)
y_true = np.reshape(test_y.values, (y_pred.shape[0],1))

acc = (y_pred - y_true).mean()
acc

In [ ]:
x = list(range(100))
plt.figure(figsize=(15,5))
plt.plot(x, y_pred[100:200], '-', label='pred')
plt.plot(x, y_true[100:200], '-', label='true')
plt.legend()
plt.show()